# Convolutional LSTM Classifier

#### Load dependencies

In [7]:
import keras
from keras.datasets import imdb
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Dropout, Embedding, SpatialDropout1D, LSTM
from keras.layers.wrappers import Bidirectional # new! 
from keras.layers import Conv1D, MaxPooling1D
from keras.callbacks import ModelCheckpoint
import os
from sklearn.metrics import roc_auc_score 
import matplotlib.pyplot as plt 
%matplotlib inline

In [8]:
#output directory name:
output_dir = 'model_output/cnnLSTM'

# training:
epochs = 4
batch_size = 128

# vector-space embedding: 
n_dim = 64 
n_unique_words = 10000 
max_review_length = 200 # both directions, so doubled it
pad_type = trunc_type = 'pre'
drop_embed = 0.2 

# convolutional layer architecture:
n_conv = 64  
k_conv = 3 
mp_size = 4

# LSTM layer architecture:
n_lstm = 64 
drop_lstm = 0.2

In [9]:
(x_train, y_train), (x_valid, y_valid) = imdb.load_data(num_words=n_unique_words) # removed n_words_to_skip

In [10]:
x_train = pad_sequences(x_train, maxlen=max_review_length, padding=pad_type, truncating=trunc_type, value=0)
x_valid = pad_sequences(x_valid, maxlen=max_review_length, padding=pad_type, truncating=trunc_type, value=0)

In [11]:
x_train[10]

array([ 764, 6075,    5, 1030,    8, 2973,   73,  469,  167, 2127,    2,
       1568,    6,   87,  841,   18,    4,   22,    4,  192,   15,   91,
          7,   12,  304,  273, 1004,    4, 1375, 1172, 2768,    2,   15,
          4,   22,  764,   55, 5773,    5,   14, 4233, 7444,    4, 1375,
        326,    7,    4, 4760, 1786,    8,  361, 1236,    8,  989,   46,
          7,    4, 2768,   45,   55,  776,    8,   79,  496,   98,   45,
        400,  301,   15,    4, 1859,    9,    4,  155,   15,   66,    2,
         84,    5,   14,   22, 1534,   15,   17,    4,  167,    2,   15,
         75,   70,  115,   66,   30,  252,    7,  618,   51,    9, 2161,
          4, 3130,    5,   14, 1525,    8, 6584,   15,    2,  165,  127,
       1921,    8,   30,  179, 2532,    4,   22,    9,  906,   18,    6,
        176,    7, 1007, 1005,    4, 1375,  114,    4,  105,   26,   32,
         55,  221,   11,   68,  205,   96,    5,    4,  192,   15,    4,
        274,  410,  220,  304,   23,   94,  205,  1

#### Architecture

In [12]:

model = Sequential()
model.add(Embedding(n_unique_words, n_dim, input_length=max_review_length)) 
model.add(SpatialDropout1D(drop_embed))
model.add(Conv1D(n_conv, k_conv, activation='relu'))
model.add(MaxPooling1D(mp_size))
model.add(Bidirectional(LSTM(n_lstm, dropout=drop_lstm)))
model.add(Dense(1, activation='sigmoid'))

Instructions for updating:
`NHWC` for data_format is deprecated, use `NWC` instead


In [13]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 200, 64)           640000    
_________________________________________________________________
spatial_dropout1d_2 (Spatial (None, 200, 64)           0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 198, 64)           12352     
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 49, 64)            0         
_________________________________________________________________
bidirectional_1 (Bidirection (None, 128)               66048     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 129       
Total params: 718,529
Trainable params: 718,529
Non-trainable params: 0
_________________________________________________________________


#### Configure model

In [14]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [15]:
modelcheckpoint = ModelCheckpoint(filepath=output_dir+"/weights.{epoch:02d}.hdf5")
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

In [16]:
model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, verbose=1, validation_data=(x_valid, y_valid), callbacks=[modelcheckpoint])

Train on 25000 samples, validate on 25000 samples
Epoch 1/4
25000/25000 [==============================] - 16s 645us/step - loss: 0.4528 - acc: 0.7622 - val_loss: 0.3133 - val_acc: 0.8718

#### Evaluate

In [17]:
model.load_weights(output_dir+"/weights.01.hdf5") # zero-indexed

In [18]:
y_hat = model.predict_proba(x_valid)

In [19]:
"{:0.2f}".format(roc_auc_score(y_valid, y_hat)*100.0)

'94.49'